The first operation in Python is to import the libraries one will be working with

In [1]:
import pandas as pd
import numpy as np
import chaospy as cp

I have taken the file Carlos provided me and made a few adjustments, most of which are minor ones, i.e. plainly format.

I have also: i) Edited the iTFA intake as gamma-shape distributions;
            ii) Considered their min, max in the range (0, 5) % daily energy int.
           iii) Furthermore distributions for the Relative risk of a 2% E iTFA intake has been considered as well (truncated normal), probability of 2nd CHD attack (1,2) (truncated gamma) and inflation rate (0,7) (truncated normal).

In [2]:
df = pd.read_excel('C:/Users/Samuele/Dropbox/ECGC_Barcelona/1.Projects/02-JRC/Work/iTFA/Parameters_distributions.xlsx')
df2 = df.sort_values(['Shape'], ascending = True)
df3 = df2[df2.columns[0:7]]
df4 = df3.reset_index()
del df4['index']
df4.loc[25]=['Inflation rate', '%', 'normal', 0, 7, 3.5, 3.5]
df4.loc[26]=['Probability of 2nd CHD attack', '', 'lognormal', 1, 2, 1.5, 1.5]
df4.loc[27]=['Relative Risk', '', 'normal', 1.11, 1.37, 1.23, (1.37-1.11)/1.96]
df4['Shape'].loc[21:24]='gamma'
df4['Min'].loc[21:24]=0
df4['Max'].loc[21:24]=5

C:\Users\Samuele\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


The workflow methodology builds on the logic to take out each distribution-shape typology and progressively narrow down the selection. Eventually, the dataframes generated are merged into a final one. Let's start by tackling the easiest case, the uniform distributions

In [5]:
unif = []
df5 = pd.DataFrame()
sample = []
if df4['Shape'].any()=='Uniform':
    for i1 in df4['Variable'].loc[df4['Shape'] == 'Uniform'].index:
        dist=cp.Uniform(df4['Min'].loc[df4['Shape'] == 'Uniform'][i1],df4['Max'].loc[df4['Shape'] == 'Uniform'][i1])
        unif.append(dist)
        unifS=unif[i1].sample(1000, rule="S")
        np.random.shuffle(unifS)
        sample.append(unifS)
        df5 = pd.DataFrame(sample, index=df4['Variable'].loc[df4['Shape'] == 'Uniform'].values)

OK, it's time for the truncated normal and the lognormal and eventually the gammas. As regards the latter, the missing-value figures for the (min,max) are filled with (0, 1e99) for the sake of praticality. Just a brief comment about the truncation I eventually developed, it can be implemented by putting in place a nested command, i.e. (cp for chaospy) cp.trunc(cp.gamma(...),...) that assures the distribution is generated within a lower and upper threshold. 

In [6]:
#The database is then readjusted and the distributions of different shape are also built 
df6=df4.loc[2:]
df7 = df6.sort_values(['Shape'], ascending = False)
df8 = df7.reset_index()
del df8['index']
normal = []
sampleN = []
if df8['Shape'].any()=='normal':
    for i2 in df8['Variable'].loc[df8['Shape'] == 'normal'].index:
        distN=cp.Truncnorm(df8['Min'].loc[df8['Shape'] == 'normal'][i2],df8['Max'].loc[df8['Shape'] == 'normal'][i2],\
        df8['Mean'].loc[df8['Shape'] == 'normal'][i2], df8['Std'].loc[df8['Shape'] == 'normal'][i2])                   
        normal.append(distN)
        normalS=normal[i2].sample(1000, rule="S")
        np.random.shuffle(normalS)
        sampleN.append(normalS)
df9 = pd.DataFrame(sampleN, index=df8['Variable'].loc[df8['Shape'] == 'normal'].values)

df8b=df8.loc[2:].reset_index()
del df8b['index']
df8c = df8b
lognormal = []
sampleL = []
if df8b['Shape'].any()=='lognormal':
    for i2b in df8c['Variable'].loc[df8c['Shape'] == 'lognormal'].index:
        distL=cp.trunk(cp.Lognormal(df8c['Mean'].loc[df8c['Shape'] == 'lognormal'][i2b],df8c['Std'].loc[df8c['Shape'] == 'lognormal'][i2b],0),\
        df8c['Max'].loc[df8c['Shape'] == 'lognormal'][i2b])        
        lognormal.append(distL)
        lognormalS=lognormal[i2b].sample(1000, rule="S")
        np.random.shuffle(lognormalS)
        sampleL.append(lognormalS)
df9b = pd.DataFrame(sampleL, index=df8c['Variable'].loc[df8c['Shape'] == 'lognormal'].values)

df11=df8.loc[4:]
df13 = df11.sort_values(['Shape'], ascending = False)
df13 = df11.reset_index()
del df13['index']
df13a = df13
gamma = []
sampleG = []
if df13['Shape'].any()=='gamma':
    df13a['Max']=df13['Max'].loc[df13['Shape'] == 'gamma'].fillna(1e99)
    df13a['Min']=df13['Min'].loc[df13['Shape'] == 'gamma'].fillna(0)    
    for i3 in df13a['Variable'].loc[df13a['Shape'] == 'gamma'].index:
        distG=cp.trunk(cp.Gamma((df13a['Mean']**2/df13a['Std']**2).loc[df13a['Shape'] == 'gamma'][i3],(df13a['Std']**2/df13a['Mean']).loc[df13a['Shape'] == 'gamma'][i3],\
        df13a['Min'].loc[df13a['Shape'] == 'gamma'][i3]),df13a['Max'].loc[df13a['Shape'] == 'gamma'][i3])
        gamma.append(distG)
        gammaS=gamma[i3].sample(1000, rule="S")
        np.random.shuffle(gammaS)
        sampleG.append(gammaS)
df16 = pd.DataFrame(sampleG, index=df13a['Variable'].loc[df13a['Shape'] == 'gamma'].values)

By concatenating the four dataframes generated, it is possible to have a final one ready for delivery.

In [7]:
dfF = pd.concat([df5, df9, df9b, df16])

Almost ready for delivery, it is better transposing the dataframe in order to have a column-based variable representation.

In [8]:
dfFi = dfF.T

Last step is to export the data to an excel file ready for use.

In [9]:
dfFi.to_excel('C:/Users/Samuele/Dropbox/ECGC_Barcelona/1.Projects/02-JRC/Work/iTFA/gamma_intake_1000.xlsx')
